# Формирование доп. набора данных

Ниже формируется набор данных, состоящий из изображений, которые содержат визуальный стимул "Выражение лица"

In [ ]:
import os
import json

# Путь до папки с аннотациями
annotation_folder = "C:\EmoSet\EmoSet-118K\annotation"

# Создание списков для каждой эмоции
emotions = ["amusement", "anger", "awe", "contentment", "disgust", "excitement", "fear", "sadness"]
emotion_lists = {emotion: [] for emotion in emotions}

# Перебор всех эмоций
for emotion in emotions:
    # Путь до папки с определенной эмоцией
    emotion_folder = os.path.join(annotation_folder, emotion)
    
    # Перебор всех файлов .json в папке эмоции
    for filename in os.listdir(emotion_folder):
        if filename.endswith(".json"):
            # Полный путь до файла
            filepath = os.path.join(emotion_folder, filename)
            # Открытие файла и загрузка данных JSON
            with open(filepath, "r") as file:
                data = json.load(file)
            # Проверка наличия ключа "facial_expression"
            if "facial_expression" in data:
                # Формирование пути до изображения
                image_id = data["image_id"]
                image_path = os.path.join("C:\EmoSet\EmoSet-118K\image", emotion, f"{image_id}.jpg")
                # Добавление пути в список соответствующей эмоции
                emotion_lists[emotion].append(image_path)


# Вывод списков для каждой эмоции
for emotion, image_paths in emotion_lists.items():
    print(f"Список изображений для эмоции {emotion}:")
    for path in image_paths:
        print(path)
    print()
    
    
print(emotion_lists)

amusement_paths = []
amusement_paths = emotion_lists["amusement"]

anger_paths = []
anger_paths = emotion_lists["anger"]

awe_paths = []
awe_paths = emotion_lists["awe"]

contentment_paths = []
contentment_paths = emotion_lists["contentment"]

disgust_paths = []
disgust_paths = emotion_lists["disgust"]

excitement_paths = []
excitement_paths = emotion_lists["excitement"]

fear_paths = []
fear_paths = emotion_lists["fear"]

sadness_paths = []
sadness_paths = emotion_lists["sadness"]

print(amusement_paths[0:5])

### Запись в файл путей до изображений

In [ ]:
# Открываем файл для записи
with open('img_paths_amusement_face_expr.txt', 'w') as f:
    # Преобразуем каждый элемент списка в строку и записываем в файл
    for item in amusement_paths:
        f.write(str(item) + '\n')

with open('img_paths_anger_face_expr.txt', 'w') as f:
    # Преобразуем каждый элемент списка в строку и записываем в файл
    for item in anger_paths:
        f.write(str(item) + '\n')
        
with open('img_paths_awe_face_expr.txt', 'w') as f:
    # Преобразуем каждый элемент списка в строку и записываем в файл
    for item in awe_paths:
        f.write(str(item) + '\n')
        
with open('img_paths_contentment_face_expr.txt', 'w') as f:
    # Преобразуем каждый элемент списка в строку и записываем в файл
    for item in contentment_paths:
        f.write(str(item) + '\n')
        
with open('img_paths_excitement_face_expr.txt', 'w') as f:
    # Преобразуем каждый элемент списка в строку и записываем в файл
    for item in excitement_paths:
        f.write(str(item) + '\n')
        
with open('img_paths_fear_face_expr.txt', 'w') as f:
    # Преобразуем каждый элемент списка в строку и записываем в файл
    for item in fear_paths:
        f.write(str(item) + '\n')
        
with open('img_paths_disgust_face_expr.txt', 'w') as f:
    # Преобразуем каждый элемент списка в строку и записываем в файл
    for item in disgust_paths:
        f.write(str(item) + '\n')
        
with open('img_paths_sadness_face_expr.txt', 'w') as f:
    # Преобразуем каждый элемент списка в строку и записываем в файл
    for item in sadness_paths:
        f.write(str(item) + '\n')

# Извлечение признаков из изображений + проставление меток

In [4]:
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
from hsemotion.facial_emotions import HSEmotionRecognizer
import torchvision.models as models
from hsemotion.facial_emotions import HSEmotionRecognizer

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
print(device)
fer = HSEmotionRecognizer(model_name='enet_b0_8_best_afew', device=device)

C:\Users\Юлия\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
C:\Users\Юлия\.hsemotion\enet_b0_8_best_afew.pt Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [5]:
from facenet_pytorch import MTCNN
mtcnn = MTCNN(keep_all=False, post_process=False, min_face_size=40, device=device)

def detect_face(frame):
    bounding_boxes, probs = mtcnn.detect(frame, landmarks=False)
#     print(probs)
    if all(pr is not None for pr in probs):
        bounding_boxes=bounding_boxes[probs>0.9]
        return bounding_boxes
    else:
        return [[-100]]

In [6]:
x_FE = []

In [7]:
labels = []

In [9]:
amusement_paths, anger_paths, awe_paths, disgust_paths, fear_paths, contentment_paths, excitement_paths, sadness_paths = [], [], [], [], [], [], [], []

In [10]:
with open('img_paths_amusement_face_expr.txt', "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        amusement_paths.extend(paths)

with open('img_paths_anger_face_expr.txt', "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        anger_paths.extend(paths)
        
with open('img_paths_awe_face_expr.txt', "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        awe_paths.extend(paths)
        
with open('img_paths_contentment_face_expr.txt', "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        contentment_paths.extend(paths)
        
with open('img_paths_excitement_face_expr.txt', "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        excitement_paths.extend(paths)
        
with open('img_paths_fear_face_expr.txt', "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        fear_paths.extend(paths)
        
with open('img_paths_disgust_face_expr.txt', "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        disgust_paths.extend(paths)
        
with open('img_paths_sadness_face_expr.txt', "r") as file:
    for line in file:
        # Удаление символа новой строки и разделение строки на пути к изображениям
        paths = line.strip().split()

        # Добавление путей к изображениям в список
        sadness_paths.extend(paths)

In [11]:
for fpath in amusement_paths:
    frame_bgr = cv2.imread(fpath)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes=detect_face(frame)
    for bbox in bounding_boxes:
        if all(coord >= 0 for coord in bbox):  
#             box = bbox.astype(int)
#             x1,y1,x2,y2=box[0:4]    
#             face_img=frame[y1:y2,x1:x2,:]
# #         emotion,scores=fer.predict_emotions(face_img,logits=True)
#             features=fer.extract_features(face_img)
#             x_FE.append(features)
            labels.append(0)
    

In [12]:
for fpath in awe_paths:
    frame_bgr = cv2.imread(fpath)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes=detect_face(frame)
    for bbox in bounding_boxes:
        if all(coord >= 0 for coord in bbox):  
#             box = bbox.astype(int)
#             x1,y1,x2,y2=box[0:4]    
#             face_img=frame[y1:y2,x1:x2,:]
# #         emotion,scores=fer.predict_emotions(face_img,logits=True)
#             features=fer.extract_features(face_img)
#             x_FE.append(features)
            labels.append(1)

In [13]:
for fpath in contentment_paths:
    frame_bgr = cv2.imread(fpath)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes=detect_face(frame)
    for bbox in bounding_boxes:
        if all(coord >= 0 for coord in bbox):  
#             box = bbox.astype(int)
#             x1,y1,x2,y2=box[0:4]    
#             face_img=frame[y1:y2,x1:x2,:]
# #         emotion,scores=fer.predict_emotions(face_img,logits=True)
#             features=fer.extract_features(face_img)
#             x_FE.append(features)
            labels.append(2)

In [14]:
for fpath in excitement_paths:
    frame_bgr = cv2.imread(fpath)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes=detect_face(frame)
    for bbox in bounding_boxes:
        if all(coord >= 0 for coord in bbox):  
            box = bbox.astype(int)
#             x1,y1,x2,y2=box[0:4]    
#             face_img=frame[y1:y2,x1:x2,:]
# #         emotion,scores=fer.predict_emotions(face_img,logits=True)
#             features=fer.extract_features(face_img)
#             x_FE.append(features)
            labels.append(3)

In [15]:
for fpath in anger_paths:
    frame_bgr = cv2.imread(fpath)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes=detect_face(frame)
    for bbox in bounding_boxes:
        if all(coord >= 0 for coord in bbox):  
#             box = bbox.astype(int)
#             x1,y1,x2,y2=box[0:4]    
#             face_img=frame[y1:y2,x1:x2,:]
# #         emotion,scores=fer.predict_emotions(face_img,logits=True)
#             features=fer.extract_features(face_img)
#             x_FE.append(features)
            labels.append(4)

In [16]:
for fpath in disgust_paths:
    frame_bgr = cv2.imread(fpath)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes=detect_face(frame)
    for bbox in bounding_boxes:
        if all(coord >= 0 for coord in bbox):  
#             box = bbox.astype(int)
#             x1,y1,x2,y2=box[0:4]    
#             face_img=frame[y1:y2,x1:x2,:]
# #         emotion,scores=fer.predict_emotions(face_img,logits=True)
#             features=fer.extract_features(face_img)
#             x_FE.append(features)
            labels.append(5)

In [17]:
for fpath in fear_paths:
    frame_bgr = cv2.imread(fpath)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes=detect_face(frame)
    for bbox in bounding_boxes:
        if all(coord >= 0 for coord in bbox):  
#             box = bbox.astype(int)
#             x1,y1,x2,y2=box[0:4]    
#             face_img=frame[y1:y2,x1:x2,:]
# #         emotion,scores=fer.predict_emotions(face_img,logits=True)
#             features=fer.extract_features(face_img)
#             x_FE.append(features)
            labels.append(6)

In [18]:
for fpath in sadness_paths:
    frame_bgr = cv2.imread(fpath)
    frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    bounding_boxes=detect_face(frame)
    for bbox in bounding_boxes:
        if all(coord >= 0 for coord in bbox):   
#             box = bbox.astype(int)
#             x1,y1,x2,y2=box[0:4]    
#             face_img=frame[y1:y2,x1:x2,:]
# #         emotion,scores=fer.predict_emotions(face_img,logits=True)
#             features=fer.extract_features(face_img)
#             x_FE.append(features)
            labels.append(7)

In [82]:
print(x_FE[0:5])

[array([[ 0.02731288, -0.00891317,  0.16020496, ...,  0.7200315 ,
         0.91487694,  0.35373706]], dtype=float32), array([[-0.11184178, -0.03431252,  0.04057299, ..., -0.07032545,
         0.26347637,  0.04450111]], dtype=float32), array([[ 0.00639499,  0.00529858,  0.0537325 , ..., -0.04761496,
         0.12591021,  0.06138283]], dtype=float32), array([[0.435474  , 0.08516692, 0.02083516, ..., 0.5775283 , 0.97313917,
        0.01348583]], dtype=float32), array([[-0.05291691,  0.18866904,  0.51094985, ...,  0.9378027 ,
         0.75901   ,  0.30904114]], dtype=float32)]


In [83]:
print(len(x_FE))
# получилось немного больше экземпляров, так как на некоторых изображениях по 2 лица
# из-за того, что на некоторых фото по 2 лица -> пришлось добавлять строчку разметки (например, labels.append(7))

35823


In [84]:
print(len(x_FE[0][0]))

1280


In [87]:
print(x_FE[0].shape)

(1, 1280)


In [20]:
print(len(labels))

35823


## Запись features в файл

In [89]:
# запись extracted features в файл

# Имя файла для записи
output_file = "x_FE_bounding_boxes_HSEmotion.txt"

# Запись всех массивов в файл
with open(output_file, "w") as file:
    for arr in x_FE:
        # Запись элементов массива в файл
        for a in arr:
            file.write('[')
            for elem in a:
                file.write(str(elem) + ", ")
            file.write(']')
        file.write("\n")

# // Начало запусков экспериментов отсюда

In [21]:
# чтение файла x_FE_HSEmotion в другой массив
import numpy as np

# Инициализация списка для хранения
X = []

# Чтение файла и заполнение списка X
with open("x_FE_bounding_boxes_HSEmotion.txt", "r") as file:
    for line in file:
        # Преобразование строки в список чисел
        line = line.strip().strip('[]').split(', ')
#         print(line)
        arr = np.array([float(num) for num in line if num != ''])

        # Решейп массива к размерности (1, 1280)
        arr = arr.reshape(1, -1)

        # Добавление массива в список
        X.append(arr)


## Заполнение массивов X и Y элементами подходящей размерности

In [22]:
x_FE_1 = []

In [23]:
# изменение элементов массива x_FE (X) (на размерность, необходимую resnet18 на входе)
for item in X:
    x = torch.tensor(item)
#     x = x.view(1, 1, 1, 1280).type(torch.float32)
#     x = x.repeat(1, 3, 1, 1)
    x = x.view(1, 1, 1280).type(torch.float32)
    x = x.repeat(3, 1, 1)
    x.to(device)
    x_FE_1.append(x)

In [24]:
Y = []

In [25]:
# изменение элементов массива labels_list (y) чтобы подать модели на вход на обучение
for item in labels:
    y = torch.tensor(item)
#     y = F.one_hot(y, 8).float()
    y.to(device)
    Y.append(y)
#     print(y)

## Деление на train, valid наборы

In [26]:
import sklearn
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [27]:
# Разделяем данные на train/test sets
x_train, x_valid, y_train, y_valid = train_test_split(x_FE_1, Y, test_size=0.2, random_state=42)

## Формирование train, valid датасетов из Dataset и Dataloader

In [28]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [29]:
dataset = CustomDataset(x_train, y_train)

In [30]:
# DataLoader для пакетной загрузки данных
batch_size = 8
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [31]:
dataset_v = CustomDataset(x_valid, y_valid)

In [32]:
# DataLoader для пакетной загрузки данных
batch_size = 8
valid_dataloader = DataLoader(dataset_v, batch_size=batch_size, shuffle=True)

In [33]:
# Пример использования DataLoader
for sample in valid_dataloader:
    print("Длина sample") # здесь длина такая, так как у нас в сэмпле содержится x и y
    print(len(sample))
    print("Пример sample:\n")
    print(sample)
    
#     print("Пример sample[0]:\n")
#     print(sample[0])
    print("Длина фичей или меток в одном sample должна быть:\n")
    print(len(sample[0]))
    print("Длина фичей или меток в одном sample должна быть:\n")
    print(sample[0][0].shape)
    break

Длина sample
2
Пример sample:

[tensor([[[[ 0.9195,  0.4226,  0.0654,  ...,  1.8032,  2.4070,  0.5786]],

         [[ 0.9195,  0.4226,  0.0654,  ...,  1.8032,  2.4070,  0.5786]],

         [[ 0.9195,  0.4226,  0.0654,  ...,  1.8032,  2.4070,  0.5786]]],


        [[[ 0.2054,  0.0133, -0.0593,  ...,  0.4443,  0.2596,  0.2700]],

         [[ 0.2054,  0.0133, -0.0593,  ...,  0.4443,  0.2596,  0.2700]],

         [[ 0.2054,  0.0133, -0.0593,  ...,  0.4443,  0.2596,  0.2700]]],


        [[[-0.0352, -0.1293, -0.1979,  ...,  0.0540, -0.1997,  0.1414]],

         [[-0.0352, -0.1293, -0.1979,  ...,  0.0540, -0.1997,  0.1414]],

         [[-0.0352, -0.1293, -0.1979,  ...,  0.0540, -0.1997,  0.1414]]],


        ...,


        [[[ 0.0635, -0.1250, -0.0455,  ...,  0.0429, -0.1130, -0.2081]],

         [[ 0.0635, -0.1250, -0.0455,  ...,  0.0429, -0.1130, -0.2081]],

         [[ 0.0635, -0.1250, -0.0455,  ...,  0.0429, -0.1130, -0.2081]]],


        [[[ 0.3901,  0.0702,  0.2389,  ...,  1.7747,  1.5

In [34]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [35]:
import copy
def test(val_loader,model,device,criterion):
    model.eval()
    epoch_val_accuracy,epoch_val_loss = 0,0 
    with torch.no_grad():
        for sample in val_loader:
            X, y = sample[0], sample[1]
            X = X.to(device)
            y = y.to(device)
            
            val_output = model(X)
            if criterion is None:
                val_loss = F.nll_loss(val_output, y, reduction='sum').item()
            else:
                val_loss = criterion(val_output, y)
            
            acc = (val_output.argmax(dim=1) == y).float().mean()
            epoch_val_accuracy += acc 
            epoch_val_loss += val_loss
    return epoch_val_accuracy/ len(val_loader),epoch_val_loss/ len(val_loader)

def train(train_loader,val_loader,model,device,n_epochs, optimizer,criterion,scheduler):
    train_accuracies,val_accuracies=[],[]
    best_acc=0
    best_model=None
    for epoch in range(n_epochs):
        epoch_loss = 0
        epoch_accuracy = 0
        model.train()
#         for sample in tqdm(train_loader):
        for sample in train_loader:
            X, y = sample[0], sample[1]
            X = X.to(device)
#             print(X)
            y = y.to(device)
            output = model(X)

            loss = criterion(output, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc = (output.argmax(dim=1) == y).float().mean()
            
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        epoch_val_accuracy,epoch_val_loss = test(val_loader,model,device,criterion)
#         print(
#             f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
#         )
        train_accuracies.append(epoch_accuracy)
        val_accuracies.append(epoch_val_accuracy)
        if best_acc<epoch_val_accuracy:
            best_acc=epoch_val_accuracy
            best_model=copy.deepcopy(model.state_dict())
        if scheduler is not None:
            scheduler.step()
    
    if best_model is not None:
        model.load_state_dict(best_model)
        print(f"Best acc:{best_acc}")
        epoch_val_accuracy,epoch_val_loss = test(val_loader,model,device,criterion)
        print(
            f"val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
        )
    else:
        print(f"No best model Best acc:{best_acc}")
    return best_model,train_accuracies,val_accuracies

# Загрузка EfficientNet и настройка

## Transfer Learning

In [36]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

print(efficientnet)

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub\PyTorch\Classification\ConvNets\image_classification\models\efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [37]:
num_classes=8
#cnn_model.classifier[-1]=nn.Linear(in_features=1280, out_features=num_classes)
# fc = nn.Sequential(
#     nn.Linear(512, 100),
#     nn.LeakyReLU(0.2),
#     nn.Linear(100, 8)
# )

classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(),
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8)
)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(efficientnet))

efficientnet.classifier = classifier

Num parameters FC
10248
Num parameters ALL
5288548


In [38]:
print(efficientnet)

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [39]:
for param in efficientnet.parameters():
    param.requires_grad = False

In [43]:
for param in efficientnet.classifier.parameters():
    param.requires_grad = True

In [44]:
print("Num parameters ALL")
print(count_parameters(efficientnet))

Num parameters ALL
10248


In [45]:
from torch.optim.lr_scheduler import StepLR

In [46]:
n_epochs = 15
efficientnet=efficientnet.to(device)
optimizer = torch.optim.Adam(efficientnet.classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler = ExponentialLR(
#     optimizer,
#     gamma = 0.6
# )
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [49]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,efficientnet,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.45413771810531616
val_loss : 1.3980 - val_acc: 0.4541


In [50]:
model_path = "fe_efficientnet_tl_final.pth"

In [52]:
torch.save(efficientnet, model_path)

## Fine Tuning

In [53]:
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

print(efficientnet)

Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
Using cache found in C:\Users\Юлия/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [54]:
num_classes=8
classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(),
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8)
)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(efficientnet))

efficientnet.classifier = classifier

Num parameters FC
10248
Num parameters ALL
5288548


In [55]:
print(efficientnet)

EfficientNet(
  (stem): Sequential(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (activation): SiLU(inplace=True)
  )
  (layers): Sequential(
    (0): Sequential(
      (block0): MBConvBlock(
        (depsep): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (se): SequentialSqueezeAndExcitation(
          (squeeze): Linear(in_features=32, out_features=8, bias=True)
          (expand): Linear(in_features=8, out_features=32, bias=True)
          (activation): SiLU(inplace=True)
          (sigmoid): Sigmoid()
          (mul_a_quantizer): Identity()
          (mul_b_quantizer): Identity()
        )
      

In [56]:
for param in efficientnet.parameters():
    param.requires_grad = True

In [57]:
print("Num parameters ALL")
print(count_parameters(efficientnet))

Num parameters ALL
4017796


In [58]:
from torch.optim.lr_scheduler import StepLR

In [59]:
n_epochs = 15
efficientnet=efficientnet.to(device)
optimizer = torch.optim.Adam(efficientnet.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)

criterion=nn.CrossEntropyLoss()

In [61]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,efficientnet,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.54618029467144973
val_loss : 1.2149 - val_acc: 0.5462


In [62]:
model_path = "fe_efficientnet_ft_final.pth"

In [63]:
torch.save(efficientnet, model_path)

# Загрузка ResNet18 и настройка

## Transfer Learning

In [64]:
from torchvision import models
# cnn_model = models.mobilenet_v2(pretrained=True)
cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [65]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [66]:
num_classes=8

fc = nn.Sequential(
    nn.Linear(512, 8)
)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
4104
Num parameters ALL
11689512


In [67]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [68]:
for param in cnn_model.parameters():
    param.requires_grad = False

In [69]:
for param in cnn_model.fc.parameters():
    param.requires_grad = True

In [70]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
4104


In [71]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
criterion=nn.CrossEntropyLoss()

In [73]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.39702492041378439
val_loss : 1.5993 - val_acc: 0.3970


In [74]:
model_path = "fe_resnet18_tl_final.pth"

In [75]:
torch.save(cnn_model, model_path)

## Fine Tuning

In [76]:
from torchvision import models
# cnn_model = models.mobilenet_v2(pretrained=True)
cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [77]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [78]:
num_classes=8

fc = nn.Sequential(
    nn.Linear(512, 8)
)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
4104
Num parameters ALL
11689512


In [79]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [80]:
for param in cnn_model.parameters():
    param.requires_grad = True

In [81]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
11180616


In [82]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
criterion=nn.CrossEntropyLoss()

In [83]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.54958639229134985
val_loss : 1.2574 - val_acc: 0.5496


In [84]:
model_path = "fe_resnet18_ft_final.pth"

In [85]:
torch.save(cnn_model, model_path)

## Загрузка MobileNet и настройка

### Transfer learning

In [86]:
from torchvision import models
cnn_model = models.mobilenet_v2(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

C:\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [87]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [88]:
num_classes=8

classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8, bias=True)
)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.classifier = classifier

Num parameters FC
10248
Num parameters ALL
3504872


In [89]:
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [90]:
for param in cnn_model.parameters():
    param.requires_grad = False
for param in cnn_model.classifier.parameters():
    param.requires_grad = True

In [91]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
10248


In [92]:
from torch.optim.lr_scheduler import StepLR

In [93]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [107]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.42830243974669758
val_loss : 1.4747 - val_acc: 0.4283


In [108]:
model_path = "fe_mobilenet_tl_final.pth"

In [109]:
torch.save(cnn_model, model_path)

### Fine tuning

In [110]:
from torchvision import models
cnn_model = models.mobilenet_v2(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [111]:
num_classes=8
classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(1280, 8, bias=True)
)

print("Num parameters FC")
print(count_parameters(classifier))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.classifier = classifier

Num parameters FC
10248
Num parameters ALL
3504872


In [112]:
print(cnn_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [113]:
for param in cnn_model.parameters():
    param.requires_grad = True

In [114]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
2234120


In [115]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [116]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.55105649347900273
val_loss : 1.1984 - val_acc: 0.5511


In [117]:
model_path = "fe_mobilenet_ft_final.pth"

In [118]:
torch.save(cnn_model, model_path)

## Загрузка ResNet50 и настройка

### Transfer learning

In [119]:
from torchvision import models
cnn_model = models.resnet50(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

C:\anaconda3\envs\pytorch_gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [120]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [121]:
num_classes=8
fc = nn.Linear(2048, 8, bias=True)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
16392
Num parameters ALL
25557032


In [122]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [123]:
for param in cnn_model.parameters():
    param.requires_grad = False
for param in cnn_model.fc.parameters():
    param.requires_grad = True

In [124]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
16392


In [125]:
from torch.optim.lr_scheduler import StepLR

In [126]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.fc.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [128]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.43109872456179487
val_loss : 1.5074 - val_acc: 0.4311


In [129]:
model_path = "fe_resnet50_tl_final.pth"

In [130]:
torch.save(cnn_model, model_path)

### Fine tuning

In [131]:
from torchvision import models
cnn_model = models.resnet50(pretrained=True)
# cnn_model = models.resnet18(weights="IMAGENET1K_V1")
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [132]:
num_classes=8

fc = nn.Linear(2048, 8, bias=True)

print("Num parameters FC")
print(count_parameters(fc))

print("Num parameters ALL")
print(count_parameters(cnn_model))

cnn_model.fc = fc

Num parameters FC
16392
Num parameters ALL
25557032


In [133]:
print(cnn_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [134]:
for param in cnn_model.parameters():
    param.requires_grad = True

In [135]:
print("Num parameters ALL")
print(count_parameters(cnn_model))

Num parameters ALL
23524424


In [136]:
n_epochs = 15
cnn_model=cnn_model.to(device)
optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, cnn_model.parameters()), lr=0.1)
scheduler = StepLR(optimizer, step_size=1, gamma=0.8)
# scheduler нужен для динамического регулирования learning rate (скорости обучения) в процессе обучения.
criterion=nn.CrossEntropyLoss()

In [137]:
best_model,train_accuracies,val_accuracies=train(train_dataloader,valid_dataloader,cnn_model,device,n_epochs, optimizer,criterion,scheduler)

Best acc:0.60750973554974312
val_loss : 1.1728 - val_acc: 0.6075


In [138]:
model_path = "fe_resnet50_ft_final.pth"

In [139]:
torch.save(cnn_model, model_path)